# Imports

In [ ]:
import numpy as np
import cv2
import os 
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

# Enviroment variables

In [ ]:
path = "../input/ocular-disease-recognition-odir5k"
datafr = pd.read_csv(os.path.join(path, "full_df.csv"))
# Image size
ROW = 224
COL = 224

# Images file names
file_names = []

# Loaded data
training_images = []
flags = []

# Features
grayscaled_images = []
inverted_images = []
thresholded_images = []
gray_histogram_of_images = []
RGB_histogram_of_images = []
conny_edged_images = []
laplacian_edged_images = []
x_edged_images = []
y_edged_images = []

threshold_mean = []
threshold_median = []
threshold_std_dev = []

conny_mean = []
conny_median = []
conny_std_dev = []

# Filtering Cataract & Healthy eyes from others

In [ ]:
cutter = 0
division = 1 # Increase to decrease number of images to load .. faster outputing for testing
file_names.clear()
flags.clear()
for label, flag, file_name in zip(datafr["Left-Diagnostic Keywords"], datafr["C"], datafr["Left-Fundus"]):
    if(("cataract" in label) and (flag == 1)):
        file_names.append(file_name)
        flags.append(1)
    elif(("normal fundus" in label) and (flag == 0)):
        if(cutter%division == 0):
            file_names.append(file_name)
            flags.append(0)
        cutter = cutter + 1

cutter = 0
for label, flag, file_name in zip(datafr["Right-Diagnostic Keywords"], datafr["C"], datafr["Right-Fundus"]):
    if(("cataract" in label) and (flag == 1)):
        file_names.append(file_name)
        flags.append(1)
    elif(("normal fundus" in label) and (flag == 0)):
        if(cutter%division == 0):
            file_names.append(file_name)
            flags.append(0)
        cutter = cutter + 1

print("Data Length =",len(file_names), "files", len(flags), "flags")

# Cataract Ratio

In [ ]:
plt.bar([0,1], [len([i for i in flags if i == 1]), len([i for i in flags if i == 0])], color = ['r', 'g'])
plt.xticks([0, 1], ['Cataract', 'Normal'])
plt.show()

# Loading Images

In [ ]:
training_images.clear()
for idx, image_name in enumerate(file_names):
    image = cv2.imread(os.path.join(path,"preprocessed_images",image_name))
    try:
        image = cv2.resize(image, (ROW, COL))
        image = cv2.normalize(image, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype =cv2.CV_8U)
        training_images.append(image)
    except:
        del flags[idx]

# Images Sample

In [ ]:
def showSamples(images, gray = False):
    figure, axes = plt.subplots(2, 2)
    axes[0, 0].title.set_text("Cataract")
    axes[0, 1].title.set_text("Normal")
    
    axes[0, 0].axis('off')
    axes[0, 1].axis('off')
    axes[1, 0].axis('off')
    axes[1, 1].axis('off')
    
    axes[0, 0].imshow(images[0],cmap='gray') if gray else axes[0, 0].imshow(cv2.cvtColor(images[0], cv2.COLOR_BGR2RGB))
    axes[0, 1].imshow(images[2],cmap='gray') if gray else axes[0, 1].imshow(cv2.cvtColor(images[2], cv2.COLOR_BGR2RGB))
    axes[1, 0].imshow(images[17],cmap='gray') if gray else axes[1, 0].imshow(cv2.cvtColor(images[17], cv2.COLOR_BGR2RGB))
    axes[1, 1].imshow(images[12],cmap='gray') if gray else axes[1, 1].imshow(cv2.cvtColor(images[12], cv2.COLOR_BGR2RGB))
    plt.show()

print(np.array(training_images).shape)
print(training_images[1961].dtype)
showSamples(training_images)

# Feature Extraction

## F1 Grayscale

In [ ]:
grayscaled_images.clear()
for idx, image in enumerate(training_images):
    gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    gray_image = cv2.normalize(src=gray_image, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    grayscaled_images.append(gray_image)

showSamples(grayscaled_images, True)

## F2 Threshold

In [ ]:
thresholded_images.clear()
for idx, image in enumerate(grayscaled_images):
    ret, image = cv2.threshold(image,127,255,cv2.THRESH_TOZERO_INV)
    thresholded_images.append(image)

showSamples(thresholded_images, True)

## F4 Grayscale Histogram

In [ ]:
gray_histogram_of_images.clear()
for idx, image in enumerate(grayscaled_images):
    image_histogram = cv2.equalizeHist(image)
    gray_histogram_of_images.append(image_histogram)

showSamples(gray_histogram_of_images, True)

## F5 RGB Histogram

In [ ]:
RGB_histogram_of_images.clear()
for idx, image in enumerate(training_images):
    R, G, B = cv2.split(image)
    image_histogram_R = cv2.equalizeHist(R)
    image_histogram_G = cv2.equalizeHist(G)
    image_histogram_B = cv2.equalizeHist(B)
    image_histogram = cv2.merge((image_histogram_R, image_histogram_G, image_histogram_B))
    RGB_histogram_of_images.append(image_histogram)

showSamples(RGB_histogram_of_images)

## F3 Conny Edge detection

In [ ]:
conny_edged_images.clear()
for idx, image in enumerate(training_images):
    image = cv2.Canny(image,30,200)
    conny_edged_images.append(image)

showSamples(conny_edged_images, True)

## F6 X Edges

In [ ]:
x_edged_images.clear()
for idx, image in enumerate(grayscaled_images):
    sobelx = cv2.Sobel(image,cv2.CV_64F,1,0,ksize=5)
    x_edged_images.append(sobelx)

showSamples(x_edged_images, True)

## F7 Y Edges

In [ ]:
y_edged_images.clear()
for idx, image in enumerate(grayscaled_images):
    sobely = cv2.Sobel(image,cv2.CV_64F,0,1,ksize=5)  
    y_edged_images.append(sobely)

showSamples(y_edged_images, True)

## F8 Laplacian Edges

In [ ]:
laplacian_edged_images.clear()
for idx, image in enumerate(grayscaled_images):
    lap_image = cv2.Laplacian(image, cv2.CV_64F)
    laplacian_edged_images.append(lap_image)

showSamples(laplacian_edged_images, True)

## F9 Mean, Median, Mode, Standard Deviation of ***Threshold***

In [ ]:
threshold_mean.clear()
threshold_median.clear()
threshold_std_dev.clear()

for idx, image in enumerate(thresholded_images):
    mean = np.mean(image)
    median = np.median(image)
    std_dev = np.std(image)
    
    threshold_mean.append(mean)
    threshold_median.append(median)
    threshold_std_dev.append(std_dev)

print(threshold_mean[0], threshold_mean[2])
print(threshold_median[0], threshold_median[2])
print(threshold_std_dev[0], threshold_std_dev[2])
print()
print(threshold_mean[17], threshold_mean[12])
print(threshold_median[17], threshold_median[12])
print(threshold_std_dev[17], threshold_std_dev[12])

## F9 Mean, Median, Mode, Standard Deviation of ***Conny***

In [ ]:
conny_mean.clear()
conny_median.clear()
conny_std_dev.clear()

for idx, image in enumerate(conny_edged_images):
    mean = np.mean(image)
    median = np.median(image)
    std_dev = np.std(image)
    
    conny_mean.append(mean)
    conny_median.append(median)
    conny_std_dev.append(std_dev)

print(conny_mean[0], conny_mean[2])
print(conny_median[0], conny_median[2])
print(conny_std_dev[0], conny_std_dev[2])
print()
print(conny_mean[17], conny_mean[12])
print(conny_median[17], conny_median[12])
print(conny_std_dev[17], conny_std_dev[12])

# Formating the data for Machine Learning

In [ ]:
training_features = np.vstack((threshold_mean,threshold_median,threshold_std_dev,conny_mean,conny_median,conny_std_dev)).T

image_train, image_test, flag_train, flag_test = train_test_split(training_features, np.asarray(flags), test_size=0.3, random_state=1)

stdSc = StandardScaler()
image_train = stdSc.fit_transform(image_train)
image_test = stdSc.transform(image_test)

image_train = np.asarray(image_train).astype('float32')
flag_train = np.asarray(flag_train).astype('float32')

image_test = np.asarray(image_test).astype('float32')
flag_test = np.asarray(flag_test).astype('float32')

# Building, Training & Testing the NN Model

In [ ]:
classifier = Sequential()

layer_info = Dense(activation='relu', input_dim=6, units=6)
classifier.add(layer_info)

layer_info = Dense(activation='relu', units=4)
classifier.add(layer_info)

layer_info = Dense(activation='sigmoid',units=1)
classifier.add(layer_info)

classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

classifier.fit(image_train, flag_train, batch_size=50, epochs=50)

flag_prediction = classifier.predict(image_test).round()

tn, fp, fn, tp = confusion_matrix(flag_test, flag_prediction).ravel()

print("True Negative =",tn)
print("False Positive =",fp)
print("False Negative =",fn)
print("True Positive =",tp)


print(confusion_matrix(flag_test, flag_prediction))
print(accuracy_score(flag_test, flag_prediction)*100)